In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cycler
import BeamDynamics as bd
import copy

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# Gamma FLASH Therapy

## Plot Drive Electron Beam at E = 35 MeV

In [ ]:
# sdfFilePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy/E35MeV_D4mm/FCCeeTargetTracking_primary.root.sdf_txt'
sdfFilePath = '../Data/Geant4/GammaFlashTherapy/WTargetSim1/E35MeV_D4mm/FCCeeTargetTracking_primary.root.sdf_txt'
driveBeam = bd.load_standard_fwf(sdfFilePath)
totElectronsIn = driveBeam.shape[0]
driveBeam.describe()

## Optimize Gamma Fluence with Target Thickness

In [ ]:
# sdfFilePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy/E35MeV_D4mm/FCCeeTargetTracking_amor_leave.root.sdf_txt'
sdfFilePath = '../Data/Geant4/GammaFlashTherapy/WTargetSim1/E35MeV_D4mm/FCCeeTargetTracking_amor_leave.root.sdf_txt'
targetOut = bd.load_standard_fwf(sdfFilePath)
targetOut.describe()

In [ ]:
# folderBasePath = '/afs/psi.ch/project/newgun/gammaFlashTherapy'
folderBasePath = '../Data/Geant4/GammaFlashTherapy/WTargetSim1'
targetThicknessList = np.arange(1, 16, 1)
driveBeamEList = np.array([5., 10., 15., 20., 25., 30., 35., 40., 45.])  # [MeV]
simCollection = []
pdgIdList = []
for driveBeamE in driveBeamEList:
    for targetThickness in targetThicknessList:
        sdfFilePath = os.path.join(
            folderBasePath,
            'E{:.0f}MeV_D{:.0f}mm/FCCeeTargetTracking_amor_leave.root.sdf_txt'.format(
                driveBeamE, targetThickness
            )
        )
        distr = bd.load_standard_fwf(sdfFilePath)
        particleCounts = {}
        EparticleType = {}
        for pdgId in distr['pdgId'].unique():
            if pdgId not in pdgIdList:
                pdgIdList.append(pdgId)
            selIds = distr['pdgId'].isin([pdgId])
            particleCounts[pdgId] = distr[selIds].shape[0]
            # TODO: Check Ekin vs. E!!!
            EparticleType[pdgId] = distr[selIds]['Ekin'].sum() / totElectronsIn / 1e3  # [J/nC]
        simCollection.append({
            'driveBeamE': driveBeamE,
            'targetThickness': targetThickness,
            'distr': distr,
            'partCounts': particleCounts,
            'EparticleType': EparticleType
        })

In [ ]:
pdgIdList

In [ ]:
partNames = {
    11: 'Electrons',
    22: 'Photons',
    -11: 'Positrons'
}

In [ ]:
def select_sims_from_collection(simCollection, selDriveBeamE, selTargetThicknesses):
    selectedSims = [sim for sim in simCollection if sim['driveBeamE'] in selDriveBeamE and sim['targetThickness'] in selTargetThicknesses]    
    partCounts = {}
    EsingleTypes = {}
    for pdgId in pdgIdList:
        partCountSingleType = []
        EsingleType = []
        for sim in selectedSims:
            try:
                partCountSingleType.append(sim['partCounts'][pdgId])
                EsingleType.append(sim['EparticleType'][pdgId])
            except KeyError:
                partCountSingleType.append(0.)
                EsingleType.append(0.)
        partCounts[pdgId] = partCountSingleType
        EsingleTypes[pdgId] = EsingleType
    return selectedSims, partCounts, EsingleTypes

In [ ]:
selTargetThicknesses = targetThicknessList[:10]
for driveBeamE in driveBeamEList:
    _, partCounts, EsingleTypes = select_sims_from_collection(simCollection, [driveBeamE], selTargetThicknesses)
    fig, ax = plt.subplots()
    axr = ax.twinx()
    for pdgId in pdgIdList:
        ax.plot(selTargetThicknesses, partCounts[pdgId], 'o-')
        axr.plot(selTargetThicknesses, EsingleTypes[pdgId], 'o--')
    ax.set_xlabel('Target thickness [mm]')
    ax.set_ylabel('Counts (solid lines)')
    axr.set_ylabel('Average E per charge [J/nC] (dashed lines)')
    ax.legend([partNames[pdgId] for pdgId in pdgIdList])
    EperChargeDriveBeam = driveBeamE / 1e3  # [J/nC]
    ax.set_title('E per charge drive beam = {:.3f} J/nC'.format(EperChargeDriveBeam))
    ax.grid()

## Energy Spectra of Different Particle Types

In [ ]:
SELECTED_TARGET_THICKNESS = 4.  # [mm]

In [ ]:
EbinWidthPhotons = .2  # [MV]
EbinsPhotons = np.arange(0., driveBeamEList[-1], EbinWidthPhotons)
photonSpectra = []
for driveBeamE in driveBeamEList:
    selectedSims, _, _ = select_sims_from_collection(simCollection, [driveBeamE], selTargetThicknesses)
    fig, ax = plt.subplots(len(pdgIdList), 1, figsize=(9.6, 9.))
    customCycler = (
        cycler.cycler(color=[plt.get_cmap('jet')(1. * ind/len(selectedSims)) for ind in range(len(selectedSims))])  # +
        # cycler(lw=[1, 2, 3, 4])
    )
    fig.suptitle('EdriveBeam = {:.1f} MeV'.format(driveBeamE))
    for ind, pdgId in enumerate(pdgIdList):
        ax[ind].set_prop_cycle(customCycler)
        for sim in selectedSims:
            countsPhotons, EbinEdgesPhotons, _ = ax[ind].hist(sim['distr'][sim['distr']['pdgId'].isin([pdgId])]['Ekin'], bins=EbinsPhotons, label='D = {:.1f} mm'.format(sim['targetThickness']))
            if pdgId == 22 and sim['targetThickness'] == SELECTED_TARGET_THICKNESS:
                photonSpectra.append(countsPhotons)
        ax[ind].set_xlabel('E [MeV]')
        ax[ind].set_ylabel('Counts')
        if ind == 0:
            ax[ind].legend()
        ax[ind].set_title(partNames[pdgId])
        ax[ind].grid()
if len(photonSpectra) != len(driveBeamEList):
    raise ValueError('Number of saved photon spectra = {:d} does not match number of simulated drive beam energies = {:d}'.format(len(photonSpectra), len(driveBeamEList)))

## Plot Photon Distribution Leaving the Target

In [ ]:
selSim, _, _ = select_sims_from_collection(simCollection, [35.], [4.])
selDistr = selSim[0]['distr']
selDistrGammas = selDistr[selDistr['pdgId'].isin([22])]

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'LongPsT']
plotDefs = bd.set_plot_defs_from_distrs([selDistrGammas], setNames=plotSets)
_ = bd.plot_distr([selDistrGammas], plotDefs)

## Convert Realistic Electron Beam Spectrum to Photon Spectrum

### Load Electron Beam Spectrum

In [ ]:
driveBeamSpectrum = np.loadtxt('../Data/Geant4/GammaFlashTherapy/PIC_sim.dat')
driveBeamSpectrum = driveBeamSpectrum.reshape((int(len(driveBeamSpectrum)/2), 2), order='F')
EbinWidthDrive = driveBeamSpectrum[1,0] - driveBeamSpectrum[0,0]  # [MeV]

In [ ]:
driveBeamSpectrumRebinned = np.zeros(driveBeamEList.shape)
EbinEdgesDrive = (driveBeamEList[1:] + driveBeamEList[:-1]) / 2.
EbinEdgesDrive = np.insert(EbinEdgesDrive, 0, 0.)
EbinEdgesDrive = np.append(EbinEdgesDrive, np.Inf)
EbinWidthDriveRebinned = EbinEdgesDrive[2] - EbinEdgesDrive[1]  ## 1st and last bins are larger
for binInd in range(len(driveBeamEList)):
    selBinInds = (EbinEdgesDrive[binInd] <= driveBeamSpectrum[:,0]) & (driveBeamSpectrum[:,0] < EbinEdgesDrive[binInd+1])
    driveBeamSpectrumRebinned[binInd] = driveBeamSpectrum[selBinInds,1].sum()
QtotBunch = driveBeamSpectrum[:,1].sum()

In [ ]:
fig, ax = plt.subplots()
ax.bar(EbinEdgesDrive[:-1], driveBeamSpectrumRebinned, width=EbinWidthDriveRebinned, align='edge')
ax.bar(driveBeamSpectrum[:,0], driveBeamSpectrum[:,1], width=-EbinWidthDrive, align='edge')
ax.set_xlabel('Electron energy [MeV]')
ax.set_ylabel('Charge [pc] / bin')
ax.legend([
    'Rebinned, Bin width = {:.2f} MeV, Total charge = {:.3f} pC'.format(EbinWidthDriveRebinned, driveBeamSpectrumRebinned.sum()),
    'Original, Bin width = {:.2f} MeV, Total charge = {:.3f} pC'.format(EbinWidthDrive, QtotBunch)
])
ax.grid()

In [ ]:
totPhotonCnts = np.zeros(photonSpectra[0].shape)
for driveBeamEInd in range(len(driveBeamEList)):
    totPhotonCnts += photonSpectra[driveBeamEInd] / totElectronsIn * driveBeamSpectrumRebinned[driveBeamEInd] / 1e12 / bd.PART_CONSTS['Q'][-11]
EbinCentersPhotons = (EbinEdgesPhotons[1:] + EbinEdgesPhotons[:-1]) / 2.
EtotPhotons = (totPhotonCnts * EbinCentersPhotons).sum() * 1e6 * bd.PART_CONSTS['Q'][-11]  # [J]

In [ ]:
fig, ax = plt.subplots()
ax.bar(EbinEdgesPhotons[:-1], totPhotonCnts, width=EbinWidthPhotons, align='edge')
# TODO: checke edges above
ax.set_xlabel('Photon energy [MV]')
ax.set_ylabel('Counts')
ax.set_title('Total energy of {:.3e} J from bunch charge of {:.3f} pC'.format(EtotPhotons, QtotBunch))
ax.grid()

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>